In [1]:
from data.data_pipelines import etl_dowaward_giordano

In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
df = etl_dowaward_giordano()

querying database.. 
2024-03-26 19:25:10,863 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 19:25:10,863 INFO sqlalchemy.engine.Engine SELECT stock_prices.id, stock_prices.ticker, stock_prices.price_date, stock_prices.open, stock_prices.close, stock_prices.adjusted_close, stock_prices.high, stock_prices.low, stock_prices.volume, stock_prices.data_source, stock_prices.update_date 
FROM stock_prices
2024-03-26 19:25:10,864 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ()


2024-03-26 19:25:11,137 INFO sqlalchemy.engine.Engine ROLLBACK
calculating returns..
calculating garch..
calculating correlations..


In [4]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [ ]:



def calculate_ATR(df):
    pass

def rank_signals(df, columns_to_rank):
    pass

In [10]:
df.set_index('price_date', inplace=True)
df['prev_close'] = df.groupby('ticker')['close'].shift(1)
tr_components =  pd.concat([
        (df['high'] - df['low']),
        (df['high'] - df['prev_close']).abs(),
        (df['low'] - df['prev_close']).abs()
        ], axis=1)

df['tr'] = tr_components.max(axis=1)


In [14]:
tests = df.groupby('ticker')['tr'].rolling(42).sum().reset_index()

In [29]:
testing = df[df.ticker=='SHY'].reset_index().merge(tests[tests.ticker == 'SHY'],on=['price_date', 'ticker'], how='left')[['tr_x', 'tr_y']].iloc[:43]

In [31]:
testing.cumsum()

,tr_x,tr_y
0,0.129997,NaN
1,0.429993,NaN
2,0.619987,NaN
3,0.859985,NaN
4,1.019989,NaN
5,1.279991,NaN
6,1.459991,NaN
7,1.619987,NaN
8,1.719986,NaN
9,1.819984,NaN


In [ ]:
# 42 period atr + highest close of 63 period
# 42 period atr - lowest low of 105 period
atr_period = 42
high_period = 63
low_period = 105
df['rolling_tr'] = df.groupby('ticker')['tr']\
    .transform(lambda x: x.rolling(atr_period).sum())
df['atr'] = df['rolling_tr'] / atr_period

df['upper_band'] = df.groupby('ticker')['prev_close']\
    .transform(lambda x: x.rolling(high_period).max())
df['lower_band'] = df.groupby('ticker')['low']\
    .transform(lambda x: x.rolling(low_period).min())



In [28]:
from IPython.display import display_html
pd.concat([pd.DataFrame(range(0, 10)), pd.DataFrame(range(0, 10)).rolling(3).sum()], axis=1)

,0,0
0,0,NaN
1,1,NaN
2,2,3.0
3,3,6.0
4,4,9.0
5,5,12.0
6,6,15.0
7,7,18.0
8,8,21.0
9,9,24.0
